# 第三回レポート課題
## 1. 最もよく使うLoss関数の一つであるSoftmax with lossレイヤーを実装してください．
特にBackwardを自分で実装してみてください。式の導出は難しいので省略してokです。

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import *
import numpy as np

class MulLayer:     # 乗算レイヤ
    def __init__(
        self) -> None:
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y # xとyをひっくり返す
        dy = dout * self.x
        
        return dx, dy
    

class AddLayer:     # 加算レイヤ
    def __init__(self) -> None:
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy
    

class Relu:     # Reluレイヤの計算グラフ
    def __init__(self) -> None:
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)    # True/False
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
class Sigmoid:
    def __init__(self) -> None:
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
    
class Affine():
    
    def __init__(self, w, b) -> None:
        self.w = w
        self.b = b
        self.x = None
        self.y = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx 
        
class SoftmaxWithLoss():
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.x)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx